## Загрузка файлов в Google Colab

 Для работы с большими файлами в Google Colab лучше добавить все файлы на Google Drive и испортировать их в ноутбук. 

In [788]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Изменим рабочую директорию, чтобы упростить путь к файлам.

In [789]:
%cd /content/drive/My Drive/VK_CoreML_Task/

/content/drive/My Drive/VK_CoreML_Task


Датасет проще загружать в виде zip-архива, поэтому в Google Colab его необходимо разархивировать.

In [790]:
# !unzip "Datasets.zip"

## Библиотеки и пакеты

Установим необходимые библиотеки: Surprise (collaborative filtering) и LightFM (Hybrid recommender system).

In [791]:
# !pip install surprise
# !pip install lightfm

Импортируем библиотеки.

In [792]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import random
import time
from random import shuffle
from collections import defaultdict
from surprise import SVD
from surprise import Dataset as Dataset_svd
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from lightfm.data import Dataset
from lightfm import LightFM
from tqdm.auto import tqdm

In [793]:
# Установим random.seed для воспроизводимости результатов
np.random.seed(42)

Установим параметры вывода данных в Pandas.

In [794]:
pd.set_option("display.min_rows", 30)
pd.set_option("display.max_columns", 30)
pd.set_option('display.width', 1000)

### Загрузка данных

Считаем данные из .csv файлов. Основной файл с взаимодействиями пользователь-фильм является очень большим. Я буду использовать только часть датасета. Количество строк задается в отдельной переменной.

In [795]:
N_ROWS_UPLOAD = 2000000
# Основной файл с взаимодействием пользователь-фильм
data_rating = pd.read_csv('rating.csv', nrows = N_ROWS_UPLOAD)
# Информация о фильмах
data_movie  = pd.read_csv('movie.csv')
# Информация о тегах, выставленных пользователями к фильмам
data_tag = pd.read_csv('tag.csv')
# Информация о релевантности определенных тегов к фильмам
data_relevance = pd.read_csv('genome_scores.csv')
# Соответствие тега и его id
data_tag_id = pd.read_csv('genome_tags.csv')
# еще один файл со ссылками на оценки на других сервисах не использовался

### Фильтрация данных

Отфильтруем фильмы, имеющие меньше 500 оценок и пользователей с менее 150 и более 5000 просмотренных фильмов. Помимо информативной нагрузки (менее 150 фильмов и менее 50 оценок фильма - недостаточно данных, более 5000 фильмов в коллекции - слишком много данных), это удобно сделать и для разделения данных на train-validation-test, чтобы в каждой части было достаточное количество данных. 

In [796]:
rating_to_reduce = data_rating.copy()
rating_to_reduce['count_movie'] = rating_to_reduce['movieId'].map(rating_to_reduce['movieId'].value_counts())
# Количество фильмов > 500
rating_movies = rating_to_reduce.loc[rating_to_reduce['count_movie'] > 500]
rating_movies['count_user'] = rating_movies['userId'].map(rating_movies['userId'].value_counts())
# Количество просмотренных фильмов пользователем от 150 до 5000
rating_mov_users = rating_movies.loc[(rating_movies['count_user'] >= 150) & 
                                     (rating_movies['count_user'] <= 5000)]
# Проверим размер данных после фильтрации
rating_movies.shape, rating_mov_users.shape

((1283820, 6), (700603, 6))

### Разбиение датасета на train-validation-test части

Параметры разбиения датасета на train-validation-test.
Датасет разбивается следующим образом:
1. Для каждого пользователя выполняется сортировка по времени (timestamp).
2. Выбирается определенное количество последних записей для validation и test частей, а остальное остается в train части.

In [797]:
# Количество записей для набора validation+test
NUMBER_OF_SAMPLES_VAL_TEST = 30
# Количество записей для test части
NUMBER_OF_SAMPLES_TEST = 15
# Количество записй для validation части
NUMBER_OF_SAMPLES_VAL = NUMBER_OF_SAMPLES_VAL_TEST - NUMBER_OF_SAMPLES_TEST
# Сортировка по пользователю и в хронологическом порядке
rating = rating_mov_users.sort_values(['userId', 'timestamp'],
              ascending = [True, True], ignore_index=True)
# Отбор данных для валидации и теста
all_test_val = rating.groupby('userId').apply(lambda x: x[-NUMBER_OF_SAMPLES_VAL_TEST:]).reset_index(level=0, drop=True)
# Отбор данных для тренировочной части
train = rating.loc[set(rating.index) - set(all_test_val.index)]
# Отбор данных для теста
test = all_test_val.groupby('userId').apply(lambda x: x[-NUMBER_OF_SAMPLES_TEST:]).reset_index(level=0, drop=True)
# Отбор данных для валидации
val = all_test_val.loc[set(all_test_val.index) - set(test.index)].sort_values(['userId', 'timestamp'],
              ascending = [True, True])

Количество различных пользователей и фильмов в тренировочной части.

In [798]:
number_of_users = train['userId'].nunique()
number_of_movies = train['movieId'].nunique()
print(f'Number of users = {number_of_users} | Number of movies = {number_of_movies}')

Number of users = 2499 | Number of movies = 1000


Расчет средней оценки для каждого пользователя

In [799]:
user_average_rating = defaultdict(float)
for user in train['userId'].unique():
  user_average_rating[user] = train[train['userId'] == user]['rating'].mean(axis=0)

## Baseline решение

In [800]:
def random_recommender(test):
  recommendations = defaultdict(list)
  for user in list(test['userId'].unique()):
    tmp_recs = list(test[test['userId'] == user]['movieId'])
    shuffle(tmp_recs)
    recommendations[user] = tmp_recs
  return recommendations

In [801]:
random_recommendations = random_recommender(test)

# Рекомендательная система на основе коллаборативной фильтрации

В следующей секции находятся полезные функции для получения рекомендаций на основе коллаборативной фильтрации.

In [802]:
def SVD_train_test_preparation(train, test):
  '''Вернет train и test для SVD'''
  reader = Reader(rating_scale=(0.5, 5))
  train_svd = train[['userId', 'movieId', 'rating']].copy()
  svd_train = Dataset_svd.load_from_df(train_svd, reader)
  trainset = svd_train.build_full_trainset()
  val_svd = test[['userId', 'movieId', 'rating']].copy()
  svd_val = Dataset_svd.load_from_df(val_svd, reader)
  valset = svd_val.build_full_trainset()
  testset = valset.build_testset()
  return trainset, testset

def SVD_make_predictions(svd, testset):
  '''Вернет все предсказанные значения из test(val)'''
  predictions = svd.test(testset)
  user_est_true = defaultdict(list)
  for uid, iid, true_r, est, _ in predictions:
      user_est_true[uid].append((iid, est, true_r))
  return user_est_true

def SVD_list_recommended(user_est_true, average_ratings):
  '''Вернет отсортированный список рекомендаций'''
  recommendations = defaultdict(list)
  for user, user_ratings in user_est_true.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    recommendations[user] = [x[0] for x in user_ratings]
  return recommendations

GridSearch своими руками (т.к. не разбирался в предопределенном сплите на train-val из билиотеки Surprise, а кросс-валидацию решил не делать исходя из хронологии)

In [803]:
# Разбиение данных на тренировочную и валидационную части
trainset_svd, valset_svd = SVD_train_test_preparation(train, val)
# epochs = 30
# result_svd_grid_search = []
# for factor in range(10, 270, 50):
#   for lr in [0.001, 0.005, 0.01, 0.05, 0.1]:
#     svd = SVD(n_factors=factor, n_epochs=epochs, lr_all=lr)
#     svd.fit(trainset_svd)
#     predictions = svd.test(valset_svd)
#     rmse = accuracy.rmse(predictions)
#     result_svd_grid_search.append((factor, lr, rmse))

In [804]:
# # Получение лучшей метрики и соответствующих ей параметров
# best_result = sorted(result_svd_grid_search, key=lambda x: x[3])
# # Размерность вектора скрытых признаков
# best_fact = best_result[0][0] # 10
# # Скорость обучения
# best_lr = best_result[0][1] # 0.01
# # Лучшая метрика
# lowest_err = best_result[0][2] # 0.76

In [805]:
best_fact = 10
best_epoch = 30
best_lr = 0.01
# Модель с лучшими параметрами
svd = SVD(n_factors=best_fact, n_epochs=best_epoch, lr_all=best_lr)
# Обучение модели
svd.fit(trainset_svd)

Ниже представлена функция для расчета основных метрик: Precision@k, Recall@k, f1-score, Average Precision.

In [806]:
def classification_metrics(recs_model, average_ratings, test, top_N=10):
  ''' Вернет основные метрики и напечатает средние по всем пользователям'''
  precisions = defaultdict(float)
  recalls = defaultdict(float)
  f1_scores = defaultdict(float)
  average_precisions = defaultdict(float)
  
  for user in list(test['userId'].unique()):
    test_user = test[test['userId'] == user]
    
    test_user['relevant'] = test_user['rating'].ge(average_ratings[user]).astype(int)
    
    number_of_relevants = sum(test_user['relevant'])
    recs_user = recs_model[user][:top_N]
    
    number_of_recs = len(recs_user)
    recs_relevant = [test_user.loc[test_user['movieId'] == rec]['relevant'].item() 
                     for rec in recs_user]
    precisions_user = []
    for i, rec in enumerate(recs_relevant, start=1):
      precisions_user.append(sum(recs_relevant[:i])/(i))
    presatk = [precisions_user[i] for i in range(len(precisions_user)) if recs_relevant[i] == 1]
    precisions[user] = sum(recs_relevant) / number_of_recs if number_of_recs != 0 else 0
    recalls[user] = sum(recs_relevant)/number_of_relevants if number_of_relevants != 0 else 0
    f1_scores[user] = (2 * precisions[user] * recalls[user] 
                       / (precisions[user] + recalls[user])
                       if (precisions[user] + recalls[user]) != 0 else 0)
    average_precisions[user] = sum(presatk)/len(presatk) if len(presatk) != 0 else 0
  print(f'Precision@{top_N}: {sum(prec for prec in precisions.values()) / len(precisions)}')
  print(f'Recall@{top_N}: {sum(rec for rec in recalls.values()) / len(recalls)}')
  print(f'f1-score: {sum(f1_score for f1_score in f1_scores.values()) / len(f1_scores)}')
  print(f'AP: {sum(av_prec for av_prec in average_precisions.values()) / len(average_precisions)}')
  return precisions, recalls, f1_scores, average_precisions

Получение метрик для рекомендаций с помощью коллаборативной фильтрации

In [807]:
_, testset_svd = SVD_train_test_preparation(train, test)
# Получение предсказаний с помощью SVD
predictions_svd = SVD_make_predictions(svd, testset_svd)
# Получение рекомендаций
svd_recommendations = SVD_list_recommended(predictions_svd, user_average_rating)

In [808]:
print('Metrics. SVD:')
(precisions_svd, 
 recalls_svd, 
 f1_scores_svd, 
 average_precisions_svd) = classification_metrics(svd_recommendations,
                                                  user_average_rating, 
                                                  test, top_N=10)

Metrics. SVD:
Precision@10: 0.5937575030012006
Recall@10: 0.809573358703014
f1-score: 0.6459663211431683
AP: 0.7696873171037121


In [809]:
print('Metrics. Random:')
random_recommendations = random_recommender(test)
(precisions_svd, 
 recalls_svd, 
 f1_scores_svd, 
 average_precisions_svd) = classification_metrics(random_recommendations,
                                                  user_average_rating, 
                                                  test, top_N=10)

Metrics. Random:
Precision@10: 0.49675870348139306
Recall@10: 0.6539858922812145
f1-score: 0.5334414680276376
AP: 0.5977604053715221


# Гибридная рекомендательная система

Предобработка данных о фильмах (жанры, год выпуска, название). В итоге, использовал только жанры. 

In [810]:
movie_tmp = data_movie[['movieId', 'title', 'genres']].copy()
# Разделение названия фильма на название и год выпуска
movie_tmp[['title', 'year']] = movie_tmp['title'].str.extract('(.*)\((\d{4})\)', expand=False)
# Получение списка жанров для каждого фильма
movie_tmp['genres'] = movie_tmp['genres'].str.split('|')
# Изменение порядка столбцов для удобства
columns_titles = ["movieId","title", 'year', 'genres']
movie_tmp = movie_tmp.reindex(columns=columns_titles)

Обработка данных о тегах, выставленных пользователями к фильмам.

In [811]:
tags = data_tag[['movieId', 'tag']].copy()
movie_tags = movie_tmp.copy()
# Добавим теги к информации о фильмах
movie_tags = movie_tags.merge(tags, how='left', on='movieId')
# Заполним пропуски соответствующей фразой
movie_tags['tag'] = movie_tags['tag'].fillna('No tags here')
# Определим уникальные теги для каждого фильма
tags_list = movie_tags.groupby('movieId')['tag'].unique()
# Добавим список тегов к каждому фильму
movie_list = movie_tmp.merge(tags_list, on='movieId', how='left')

Поиск релевантных тегов.

In [812]:
movie = movie_list.copy()
relevance_tag_tmp = data_relevance.copy()
map_tag_id = data_tag_id.copy()
# Задается процент релевантности
RELEVANCE_PERCENT = 0.9
# Отбор релевантных тегов
relevance_best_tmp = relevance_tag_tmp[relevance_tag_tmp['relevance'] > RELEVANCE_PERCENT]
relevance_best_tmp = relevance_best_tmp.merge(map_tag_id, how='left', on='tagId')
relevance_best_tmp.rename(columns = {'tag':'tag_relevant'}, inplace = True)
# Определение уникальных релевантных тегов для каждого фильма
best_tags = relevance_best_tmp.groupby('movieId')['tag_relevant'].unique()
# Добавление релевантных тегов к информации о фильмах
movie = movie.merge(best_tags, on='movieId', how='left')
isnull = movie['tag_relevant'].isnull()
# Отметим, если релевантных тегов для фильма нет
movie.loc[isnull, 'tag_relevant'] = (
    movie.loc[isnull, 'tag_relevant'].apply(lambda x: ['No relevant tags']))

Определение датасета с помощью встроенного класса от LightFM

In [813]:
dataset = Dataset()
dataset.fit(train['userId'].unique(),
            train['movieId'].unique())

Бинаризация данных. Необходима для добавления положительных и отрицательных взаимодействий пользователя с фильмами. 

In [814]:
# Положительное взаимодействие - рейтинг больше 4, негативное - меньше 4
train['new_rating'] = np.where(train['rating'] >= 4, 1, -1)

Создание разряженной матрицы взаимодействий и весов.

In [815]:
(interactions, weights) = dataset.build_interactions([(x[0], x[1], x[2]) for x in train[['userId', 'movieId', 'new_rating']].values])

Количество пользователей и фильмов.

In [816]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 2499, num_items 1000.


Создание признаков фильмов (жанры + релевантные тэги).

In [817]:
movie['year'] = movie['year'].fillna('year_unknown')
year_features = movie['year'].unique()
genres_features = movie['genres'].explode().unique()
tags_relevant_features = movie['tag_relevant'].explode().unique()
movie_features = np.append(genres_features, tags_relevant_features)
dataset.fit_partial(item_features = movie_features)

Соответствие индексов из исходного датасета и внутренних индексов в датасете от LightFM.

In [818]:
lightfm_mapping = dataset.mapping()
lightfm_mapping = {
    'users_mapping': lightfm_mapping[0],
    'user_features_mapping': lightfm_mapping[1],
    'movies_mapping': lightfm_mapping[2],
    'movie_features_mapping': lightfm_mapping[3],
}
print('users_mapping len - ', len(lightfm_mapping['users_mapping']))
print('user_features_mapping len - ', len(lightfm_mapping['user_features_mapping']))
print('movies_mapping len - ', len(lightfm_mapping['movies_mapping']))
print('Users movies_features_mapping len - ', len(lightfm_mapping['movie_features_mapping']))

users_mapping len -  2499
user_features_mapping len -  2499
movies_mapping len -  1000
Users movies_features_mapping len -  2077


Обратное отображение для восстановления данных.

In [819]:
lightfm_mapping['users_inv_mapping'] = {v: k for k, v in lightfm_mapping['users_mapping'].items()}
lightfm_mapping['movies_inv_mapping'] = {v: k for k, v in lightfm_mapping['movies_mapping'].items()}

Функции для получения генераторов данных, необходимых для дальнейшей работы с моделью.

In [820]:
def df_to_tuple_iterator(df):
    return zip(*df.values.T)

def concat_last_to_list(t):
    return (t[0], list(t[1:])[0])

def df_to_tuple_list_iterator(df):
    return map(concat_last_to_list, zip(*df.values.T))

Получение разряженной матрицы взаимодействий и весов на основе тренировочного набора данных.

In [821]:
train_mat, train_mat_weights = dataset.build_interactions(df_to_tuple_iterator(train[['userId', 'movieId', 'new_rating']]))

Получение признаков на основе жанров и релевантных тегов.

In [822]:
movie['genres'] = movie['genres'].apply(lambda x: list(x))
movie['tag_relevant'] = movie['tag_relevant'].apply(lambda x: list(x))
movie['features'] = movie['genres'] + movie['tag_relevant']

Учет только тех фильмов, что есть в тренировочном наборе данных.

In [823]:
known_movie_filter = movie['movieId'].isin(train['movieId'].unique())
train_movie_features = dataset.build_item_features(
    df_to_tuple_list_iterator(
        movie.loc[known_movie_filter, ['movieId', 'features']]
    )
)

In [824]:
def hybrid_list_recommended(lightfm_mapping, test, ranks):
  '''Выдаст рекомендации с помощью гибридной модели.'''
  recommendations = defaultdict(list)
  for user in list(test['userId'].unique()):
    max_num_recs = test[test['userId'] == user].shape[0]
    row_id = lightfm_mapping['users_mapping'][user]
    temp = list(test[test.userId == user].movieId.values)
    res = list(map(lambda x: lightfm_mapping['movies_mapping'][x], temp))
    result = []
    for movie_rank in res:
      result.append((lightfm_mapping['movies_inv_mapping'][movie_rank], ranks[row_id, movie_rank]))
    recs_tmp = sorted(result, key=lambda x: x[1], reverse=True)
    recommendations[user] = [x[0] for x in recs_tmp]
  return recommendations

In [825]:
def ap_gridsearch(recs_model, average_ratings, test, top_N=10):
  '''Выдаст среднее значение Precision@k для Gridsearсh'''
  average_precisions = defaultdict(float)
  for user in list(test['userId'].unique()):
    test_user = test[test['userId'] == user]
    test_user['relevant'] = test_user['rating'].ge(average_ratings[user]).astype(int)
    recs_user = recs_model[user][:top_N]
    recs_relevant = [test_user.loc[test_user['movieId'] == rec]['relevant'].item() 
                     for rec in recs_user]
    precisions_user = []
    for i, rec in enumerate(recs_relevant, start=1):
      precisions_user.append(sum(recs_relevant[:i])/(i))
    presatk = [precisions_user[i] for i in range(len(precisions_user)) if recs_relevant[i] == 1]
    average_precisions[user] = sum(presatk)/len(presatk) if len(presatk) != 0 else 0
  print(f'AP: {sum(av_prec for av_prec in average_precisions.values()) / len(average_precisions)}')
  return sum(av_prec for av_prec in average_precisions.values()) / len(average_precisions)

Подбор гиперпараметров модели.

In [826]:
# hybrid_ap = []
# for compon in range(50, 110, 50):
#   for lr in [0.05, 0.15, 0.5]:
#     for epoch in [7]:
#       lfm_model = LightFM(no_components=compon, 
#                           learning_rate=lr,
#                           loss='logistic')
#       for _ in tqdm(range(epoch), total=epoch):
#           lfm_model.fit_partial(
#               train_mat,
#               sample_weight=train_mat_weights,
#               item_features=train_movie_features,
#               num_threads=4
#           )
#       val_mat, val_mat_weights = dataset.build_interactions(df_to_tuple_iterator(val[['userId', 'movieId']]))
#       ranks = lfm_model.predict_rank(val_mat,
#                                     train_interactions=train_mat,
#                                     item_features=train_movie_features)
#       recommendations_grid = hybrid_list_recommended(lightfm_mapping, val, ranks)
#       score = ap_gridsearch(recommendations_grid, user_average_rating, val)
#       hybrid_ap.append((compon, epoch, lr, score))


Выбор лучших параметров.

In [827]:
# # Получение лучшей метрики и соответствующих ей параметрам
# best_result = sorted(hybrid_ap, key=lambda x: x[3], reverse=True)
# # Размерность вектора скрытых признаков
# best_compon = best_result[0][0] # 50
# # Количество эпох обучения стохастического градиентного спуска
# best_epoch = best_result[0][1] # 7
# # Скорость обучения
# best_lr = best_result[0][2] # 0.5
# # Лучшая метрика
# biggest_err = best_result[0][3] # 0.62
# best_compon

Обучение модели с лучшими параметрами.

In [828]:
best_compon = 50
best_epoch = 7
best_lr = 0.5

# Модель с лучшими параметрами
lfm_model = LightFM(no_components=best_compon, 
                          learning_rate=best_lr,
                          loss='logistic')
# Обучение модели
for _ in tqdm(range(best_epoch), total=best_epoch):
          lfm_model.fit_partial(
              train_mat,
              sample_weight=train_mat_weights,
              item_features=train_movie_features,
              num_threads=4)

  0%|          | 0/7 [00:00<?, ?it/s]

Рекомендации на тестовом наборе данных.

In [829]:
test_mat, test_mat_weights = dataset.build_interactions(df_to_tuple_iterator(test[['userId', 'movieId']]))
ranks = lfm_model.predict_rank(test_mat,
                              train_interactions=train_mat,
                              item_features=train_movie_features)
recommendations_hybrid = hybrid_list_recommended(lightfm_mapping, test, ranks)

Метрики полученных рекомендаций. 

In [831]:
print('Metrics. Hybrid:')
(precisions_hybrid, 
 recalls_hybrid, 
 f1_scores_hybrid, 
 average_precisions_hybrid) = classification_metrics(recommendations_hybrid, 
                                                     user_average_rating, 
                                                     test, top_N=10)

Metrics. Hybrid:
Precision@10: 0.5217286914765902
Recall@10: 0.6922994505827659
f1-score: 0.5621251365849366
AP: 0.6395548471341174
